# Instalação do software e introdução à biblioteca pySMT

Nesta UC vamos estudar metodologias e ferramentas para modelar e verificar propriedades lógicas de sistemas.
As ferramentas que vamos usar são [SMT solvers](https://en.wikipedia.org/wiki/Satisfiability_modulo_theories#SMT_solvers) e a ferramenta de programação inteira [SCIP](https://www.scipopt.org).

A linguagem de programação que vamos usar é o [Python](https://www.python.org) e as aulas práticas serão desenvolvidas dentro de um [Jupyter](https://jupyter.org) notebook, a ser executado na plataforma
[Anaconda](https://www.anaconda.com).
Usaremos a biblioteca [OR-Tools](https://developers.google.com/optimization) para fazer a interface para o SCIP, a biblioteca [pySMT](https://github.com/pysmt/pysmt) para fazer a interface com os SMT solvers, e a biblioteca de grafos [NetworkX](https://networkx.org).

# Instalação do software


Os passos que a seguir se apresentam são para instalar o Anaconda, criar um "environment" (logica) específico para esta UC onde instalamos o Python 3.10, o Jupyter, a biblioteca pySMT, o SMT solver Z3, e as bibliotecas OR-Tools e NetworkX.

Estes passos correspondem à instalação em MacOS.

1. Instalar o Anaconda a partir do [site](https://www.anaconda.com).

2. [Iniciar o Anaconda Prompt](https://docs.anaconda.com/free/anaconda/install/verify-install/#conda).
        
3. Criar um ambiente específico (chamado "logica").

        conda create -n logica python=3.10  
        
4. Ativar o ambiente "logica".

        conda activate logica
        
5. Instalar o Jupyter nesse ambiente.
  
        conda install jupyter

6. Instalar a biblioteca pySMT e os SMT solvers (Z3 e MathSAT) nesse ambiente.

        pip install pysmt
        pysmt-install --z3
        
7. Instalar as bibliotecas OR-Tools e NetworkX nesse ambiente.

        pip install ortools
        pip install networkx
        

Para arrancar com o Jupyter, na linha de comando, fazer `jupyter notebook`. Será possível aceder às diretorias locais, e abrir os notebooks (extensão `.ipynb`) com as fichas práticas. Em sessões posteriores, basta ativar o ambiente criado utilizando novamente o comando `conda activate logica`, e lançar o jupyter a partir daí.

# Alternativa na cloud

Em alternativa à distribuição Anaconda, é possível utilizar uma solução na cloud.

1. Abrir o [Google Colab](https://colab.research.google.com/), utilizando uma conta Google.

1. Fazer _upload_ de um notebook `.ipynb`.

Neste caso, é atribuído um ambiente de execução em cada início da sessão, que é posteriormente encerrado quando termina a atividade. Como tal, é necessário fazer as instalações necessárias em cada sessão, correndo numa célula um ou vários dos comandos seguintes:

        !pip install pysmt
        !pysmt-install --z3
        !pip install ortools
        !pip install networkx

# Breve introdução à utilização de SMT solvers com  a biblioteca pySMT

O problema SMT (*Satisfiability Modulo Theories*) é o problema de satisfatibilidade para lógica de primeira ordem
no âmbito de alguma teoria lógica específica - uma teoria lógica que fixa as interpretações de certos predicados e símbolos de função. Dito de outra forma, restringe-se a satisfatibilidade a uma classe específica de modelos, numa lógica de primeira ordem tipificada e com igualdade.
Os *SMT solvers* são ferramentas que visam responder ao problema SMT. Como o problema não é decidível, pode ser necessário (ou conveniente) restringir a classe de fórmulas em consideração a um fragmento (isto é, restrição sintática) adequado.

Os SMT solvers são o motor central de muitas ferramentas de análise e verificação de programas, geração de casos de teste, bounded model checking of SW, planeamento, etc.
Existem muitos SMT solvers disponíveis. Por exemplo: Z3, MathSAT, CVC4, Yices, entre outros. Alguns são direcionados a teorias específicas;
muitos suportam o formato SMT-LIB (um formato textural normalizado de input/output para SMT solvers);
muitos fornecem recursos não padronizados. Mais informação [aqui](https://en.wikipedia.org/wiki/Satisfiability_modulo_theories#SMT_solvers). Nesta UC vamos utilizar apenas o Z3.

## A biblioteca pySMT

A biblioteca [pySMT](https://github.com/pysmt/pysmt) permite que um programa em Python comunique com vários SMT solvers, tendo por base uma linguagem comum. Permite assim codificar um problema de forma independente do SMT solver, e correr o mesmo problema com vários SMT solvers.
A documentação do pySMT pode ser encontrada em https://pysmt.readthedocs.io/en/latest/index.html.

Vamos explorar alguns exemplos disponibilizados no manual do pySMT e propor novos desafios.

## Primeiros exemplos


O pySMT é altamente estruturado, mas oferece uma API simplificada que disponibiliza as funcionalidades para a utilização usual de um SMT solver. Essa API agrupa num único módulo todas as funções para construir fórmulas, verificar a satisfatibilidade e recuperar instâncias do solver. Esse módulo é o `pysmt.shortcuts`.

Neste primeiro exemplo vamos testar a satisfatibilidade de duas fórmulas proposicionais: $(A \wedge \neg B)$ e $(A \wedge \neg A)$.

Para isso, primeiro precisamos criar duas novas variáveis $A$ e $B$. As variáveis PySMT são chamadas de “símbolos” e são criadas usando a função `Symbol()` que recebe como entrada um nome de variável e, opcionalmente, um tipo. Por omissão, os símbolos são Booleanos.

Para este exemplo, vamos precisar das seguintes funções: `Symbol`, `And`, `Not`, `is_sat` e `get_model`.

In [1]:
from pysmt.shortcuts import Symbol, And, Not, is_sat, get_model

varA = Symbol("A")    # Default type is Boolean
varB = Symbol("B")
f = And(varA, Not(varB))

res = is_sat(f)
print("f := %s is SAT? %s" % (f, res))

f := (A & (! B)) is SAT? True


O teste de satisfatibilidade da fórmula pode ser feito com a função `is_sat()`. É possivel explicitar o SMT solver que queremos usar.

In [2]:
resZ3 = is_sat(f,solver_name="z3")

print("f := %s is SAT (z3)? %s" % (f, resZ3))

f := (A & (! B)) is SAT (z3)? True


Como a fórmula é satisfazível, isso significa que existe uma interpretação para seus símbolos não lógicos que torna a fórmula verdadeira. Ou seja, que existe um modelo para a fórmula.

Para sabermos qual o modelo que o solver encontrou podemos usar a função `get_model()`. Se a fórmula é satifazível, esta função devolve um modelo para a fórmula (isto é, uma espécie de dicionário que mapeia cada variável lógica no seu valor), caso contrário, devolve `None`.

In [3]:
print("Model:")
model = get_model(f)
print(model)

Model:
B := False
A := True


Vamos agora gerar a fórmula $A \wedge \neg A$ ilustrando como podemos fazer uma substituição com o método `substitute()`. Neste caso vamos substituir a variável $B$ por $A$ na fórmula `f`.

In [4]:
g = f.substitute({varB:varA})

res = is_sat(g)
print("g := %s is SAT? %s" % (g, res))

print(get_model(g))

g := (A & (! A)) is SAT? False
None



Vamos agora trabalhar com a teoria dos inteiros, para saber se é possível arranjar valores inteiros $x$ e $y$ entre 1 e 10, tal que $x+y > 10$ e $x-y\leq 5$.

Para criar variáveis inteiras temos que indicar o seu tipo. Os tipos estão definidos no módulo `pysmt.typing` de onde temos que importar o tipo `INT`.

In [5]:
from pysmt.shortcuts import Symbol, is_sat, get_model, And
from pysmt.typing import INT

x = Symbol("x", INT)
y = Symbol("y", INT)

formula = And(1<=x , x<=10 , 1<=y , y<=10 , x+y>10 , x-y<=5)

print(get_model(formula))

y := 3
x := 8


Ao importar `pysmt.shortcuts` a notação infixa fica disponível. No entanto, podemos usar os operadores textuais importando-os de `pysmt.shortcuts`. Isto por vezes torna o código mais claro, distingindo entre os operadores do Python e do SMT.

In [6]:
from pysmt.shortcuts import Symbol, is_sat, get_model, And, LE, GE, GT, Int, Not, Or, Equals
from pysmt.typing import INT

x = Symbol("x", INT)
y = Symbol("y", INT)

formula = And(LE(Int(1),x) , GE(Int(10),x) , LE(Int(1),y) , GT(x+y,Int(10)), LE(x-y,Int(5)))

print(get_model(formula))

y := 3
x := 8


### Exercício 1

Será que esta é a única solução para este problema? Como poderiamos tirar partido do solver para saber isso?

In [7]:
# completar

formula = And(LE(Int(1),x) , GE(Int(10),x) , LE(Int(1),y) , GT(x+y,Int(10)), LE(x-y,Int(5)), Not(Equals(x,y)))

print(get_model(formula))

y := 6
x := 5


Em vez de definir uma variável de cada vez, podemos usar as listas por compreensão do Python para definir vários símbolos.
As compreensões são tão comuns no pySMT que operadores n-ários (como `And()`, `Or()`, `Plus()`) podem aceitar um objecto iterável (por exemplo, listas ou gerador). Vejamos o seguinte exemplo.

## Hello World

O problema é o seguinte:
queremos associar a cada  uma das letra que compõem as palavras HELLO e WORLD, um valor inteiro entre 1 e 10, de forma a que `H+E+L+L+O = W+O+R+L+D = 25`. Será que isso é possivel?

Vejamos a seguinte formalização do problema.

In [8]:
from pysmt.shortcuts import Symbol, LE, GE, Int, And, Equals, Plus, Solver, is_sat, get_model
from pysmt.typing import INT

hello = [Symbol(s, INT) for s in "hello"]
world = [Symbol(s, INT) for s in "world"]

letters = set(hello+world)
print(letters)

domains = And(And(LE(Int(1), l),
                  GE(Int(10), l)) for l in letters)

sum_hello = Plus(hello)
sum_world = Plus(world)

problem = And(Equals(sum_hello, sum_world),
              Equals(sum_hello, Int(25)))

formula = And(domains, problem)

print(formula)

{l, o, w, r, d, h, e}
((((1 <= l) & (l <= 10)) & ((1 <= o) & (o <= 10)) & ((1 <= w) & (w <= 10)) & ((1 <= r) & (r <= 10)) & ((1 <= d) & (d <= 10)) & ((1 <= h) & (h <= 10)) & ((1 <= e) & (e <= 10))) & (((h + e + l + l + o) = (w + o + r + l + d)) & ((h + e + l + l + o) = 25)))


Se a fórmula for muito grande, certas subfórmulas podem ser mostradas como `...`. Se quiser garantir que vê sempre a fórmula toda use o método `serialize()`.

In [9]:
print("Serialization of the formula:")
print(formula.serialize())

Serialization of the formula:
((((1 <= l) & (l <= 10)) & ((1 <= o) & (o <= 10)) & ((1 <= w) & (w <= 10)) & ((1 <= r) & (r <= 10)) & ((1 <= d) & (d <= 10)) & ((1 <= h) & (h <= 10)) & ((1 <= e) & (e <= 10))) & (((h + e + l + l + o) = (w + o + r + l + d)) & ((h + e + l + l + o) = 25)))


### Exercício 2

Veja se o problema tem solução e, se tiver, apresente uma solução.

In [10]:
# completar

print(is_sat(formula))
m = get_model(formula)
print(m)

True
o := 1
l := 2
e := 10
h := 10
d := 2
r := 10
w := 10


Estas funções de atalho são muito úteis em situações pontuais. Contudo, a forma mais usual de utilização de um SMT solver consiste em criar uma instância de um `Solver` e trabalhar com ele de forma incremental. Isto faz-se através da função `Solver()`, e
pode ser usado dentro de um contexto (com a instrução `with`) para lidar automaticamente com a destruição do solver e dos recursos associados.
É possível especificar qual o solver que queremos executar e/ou a teoria lógica em que queremos trabalhar.

Depois de criar o solver, podemos adicionar restrições de forma incremental (com o método `add_assertion()`), testar a satisfatibilidade desse conjunto de restrições (com o método `solve()`), inspecionar o modelo, etc.
No exemplo abaixo lançamos o solver Z3 e, em primeiro lugar, verificamos se a fórmula `domain` é satisfazível. Depois, em caso afirmativo, continuamos a resolver o problema. Repare que, neste exemplo, acedemos ao valor de cada símbolo com o método `get_value()`. Porém, também podemos obter o modelo usando o método `get_model()`.

In [11]:
with Solver(name="z3") as solver:
    solver.add_assertion(domains)
    if not solver.solve():
        print("Domain is not SAT!!!")
        exit()
    solver.add_assertion(problem)
    if solver.solve():
        for l in letters:
            print("%s = %s" %(l, solver.get_value(l)))
    else:
        print("No solution found")

l = 1
o = 10
w = 1
r = 3
d = 10
h = 3
e = 10


### Exercício 3

Altere o código acima de forma a usar o método `get_model()`, apresentado o resultado com o mesmo formato.

In [12]:
# completar
with Solver(name="z3") as solver:
    solver.add_assertion(domains)
    if not solver.solve():
        print("Domain is not SAT!!!")
        exit()
    solver.add_assertion(problem)
    if solver.solve():
        m = solver.get_model()
        for l in letters:
          print("%s = %s" % (l, m.get_value(l)))
    else:
        print("No solution found")

l = 1
o = 10
w = 1
r = 3
d = 10
h = 3
e = 10


Podemos lançar um solver simplemente com a indicação da teoria lógica com que queremos trabalhar.
Neste caso, o solver é escolhido entre os solvers instalados que suportam essa lógica. Se não existir é gerada uma
exceção (`NoSolverAvailableErro`).
É claro que também podemos indicar o solver.

No exemplo a seguir escolhe-se a lógica `QF_LIA` (*Quantifier-Free Linear Integer Arithmetic*).

In [13]:
from pysmt.shortcuts import Symbol, LE, GE, Int, And, Equals, Plus, Solver
from pysmt.typing import INT

hello = [Symbol(s, INT) for s in "hello"]
world = [Symbol(s, INT) for s in "world"]

letters = set(hello+world)

domains = And([And(LE(Int(1), l),
                   GE(Int(10), l)) for l in letters])

sum_hello = Plus(hello)
sum_world = Plus(world)

problem = And(Equals(sum_hello, sum_world),
              Equals(sum_hello, Int(36)))

formula = And(domains, problem)

with Solver(logic="QF_LIA") as solver:
    solver.add_assertion(domains)
    if not solver.solve():
        print("Domain is not SAT!!!")
        exit()
    solver.add_assertion(problem)
    if solver.solve():
        for l in letters:
            print("%s = %s" %(l, solver.get_value(l)))
    else:
        print("No solution found")

l = 10
o = 10
w = 10
r = 5
d = 1
h = 5
e = 1


### Exercício 4

O Cryptarithms é um jogo que consiste numa equação matemática entre números desconhecidos, cujos dígitos são representados por letras. Cada letra deve representar um dígito diferente e o dígito inicial de um número com vários dígitos não deve ser zero.

Queremos saber os dígitos a que correspondem as letras envolvidas na seguinte equação:
```
TWO + TWO = FOUR
```
Podemos modelar o problema numa teoria de inteiros. Cada letra dá origem a uma variável inteira ($T$,$W$,$O$,$F$,$U$, e $R$) e para representar a equação acima representamos cada parcela por uma expressão aritmética onde cada letra é multiplicada pelo seu “peso específico” (em base 10).

Resolver este problema equivale a resolver o seguinte sistema de equações:
$$
\left\{
\begin{array}{l}
0 \le T \le 9\\
\cdots\\
0 \le R \le 9\\
T \neq W \neq O \neq F \neq U \neq R \\
T \neq 0\\
F \neq 0\\
(100 \times T + 10 \times W + O) + (100 \times T + 10 \times W + O) = 1000 \times F + 100 \times O + 10 \times U + R
\end{array}
\right.
$$

Use o Z3 para resolver este problema. Nota: poderá ser útil usar o operador `AllDifferent` e a função `Times`.

In [14]:
# completar

from pysmt.shortcuts import Times, AllDifferent

two = [Symbol(s, INT) for s in "TWO"]
four = [Symbol(s, INT) for s in "FOUR"]


letters = set(two+four)
range = And([And(LE(Int(0), l),
                   GE(Int(9), l)) for l in letters])

words = [two, four]
nonzero =  And([Not(Equals(w[0],Int(0))) for w in words])

domains = And(range, nonzero)


def word_eval(word):
    ndigits = len(word)
    total_value = Int(0)
    for i,l in enumerate(word[::-1]):
        factor = int(10**i)
        value = Times(l, Int(factor))
        total_value = Plus(total_value, value)
    return total_value


with Solver(name="z3") as solver:
    solver.add_assertion(domains)
    solver.add_assertion(AllDifferent(letters))
    if not solver.solve():
        print("Domain is not SAT!!!")
        exit()

    LHS = Times(word_eval(two), Int(2))
    # LHS = Plus(LHS,LHS)
    RHS = word_eval(four)
    # eq = Equals(word_eval(two, solver),Int(10))
    eq = Equals(LHS,RHS)
    solver.add_assertion(eq)

    if solver.solve():
        for l in letters:
            print("%s = %s" %(l, solver.get_value(l)))
    else:
        print("No solution found")

T = 9
W = 3
O = 8
F = 1
U = 7
R = 6


### Exercício 5

Considere o seguinte enigma:
```
- If the unicorn is mythical, then it is immortal.
- If the unicorn is not mythical, then it is a mortal mammal.
- If the unicorn is either immortal or a mammal, then it is horned.
- The unicorn is magical if it is horned.

Given these constraints:
   - Is the unicorn magical?
   - Is it horned?
   - Is it mythical?
```
Modele o problema em lógica proposicional criando uma variável proposicional para cada caracteristica dos unicornios.
Use um SMT solver para o resolver.

**Sugestão:** Resolva o problema com o auxílio de 5 variáveis proposicionais, correspondentes às 5 propriedades dos unicórnios. Relembre que a afirmação $A_1, \ldots, A_n \models B$ é válida se e só se o conjunto de restrições $\{A_1, \ldots, A_n, \neg B\}$ é inconsistente. Tire proveito dos métodos `push()` e `pop()` para responder às várias questões usando de forma incremental o mesmo solver.

In [15]:
# completar
from pysmt.shortcuts import BOOL, EqualsOrIff, Implies

properties = [Symbol(p, BOOL) \
              for p in ["mythical", "immortal", "mammal", "horned", "magical"]]

def get_premises():
    One = Implies(properties[0], properties[1])
    Two = Implies(Not(properties[0]), properties[2])
    Three = Implies(Or(properties[1], properties[2]), properties[3])
    Four = EqualsOrIff(properties[3], properties[4])
    premises = [One, Two, Three, Four]
    return premises

def is_implied(prop, solver, vars):
    solver.push()
    solver.add_assertion(Not(prop))
    if solver.solve():
        print(f"> The premises imply that unicorns are not {prop}.")
        for v in vars:
            print("%s = %s" %(v, solver.get_value(v)))
    else:
        print(f"> The premises imply that unicorns are {prop}.")
    solver.pop()

with Solver(name="z3") as solver:
    premises = get_premises()
    for p in premises:
        solver.add_assertion(p)

    to_check = [properties[i] for i in [4, 3, 0]]
    for prop in to_check:
        is_implied(prop, solver, properties)

> The premises imply that unicorns are magical.
> The premises imply that unicorns are horned.
> The premises imply that unicorns are not mythical.
mythical = False
immortal = False
mammal = True
horned = True
magical = True


## Breve introdução à utilização do SCIP em Python

A documentação do OR-Tools pode ser encontrada em https://developers.google.com/optimization/introduction/overview.

Para resolver um problema de programação inteira com esta biblioteca é necessário seguir uma série de passos:
1. Importar a biblioteca de programação linear do OR-Tools usando o comando `from ortools.linear_solver import pywraplp`
1. Criar uma instância do *solver* com o método `pywraplp.Solver.CreateSolver('SCIP')`.
1. Adicionar as variáveis do problema. Para criar uma variável inteira deve ser usado o método `IntVar` que recebe 3 parâmetros: o limite inferior, o limite superior, e o nome da variável. Na definição dos limites pode ser usada a constante `solver.infinity()`. O método `NumVar` pode ser usado para criar uma variável contínua, e o método `BoolVar` para criar uma variável inteira binária. Neste último caso, o método só tem um parâmetro que é o nome da variável, sendo os limites pré-definidos como 0 e 1.
1. Adicionar as restrições do problema usando o método `Add`. A restrição é definida com a sintaxe normal Python, podendo também ser usada a função `sum` para fazer o somatório de uma lista de expressões aritméticas.
1. Definir o objectivo do problema com os métodos `Maximize` ou `Minimize`, que recebem como parâmetro a função objectivo. Mais uma vez a função objectivo é definida com a sintaxe usual do Python. Este passo é opcional: se não for definido um objectivo será calculada uma qualquer solução.
1. Invocar o solver com o método `Solve`. Este método pode devolver um dos seguintes códigos:
  - `pywraplp.Solver.OPTIMAL`, quando é possível resolver o problema.
  - `pywraplp.Solver.INFEASIBLE`, quando não é possível resolver o problema.
  - `pywraplp.Solver.UNBOUNDED`, quando a solução não está limitada superiormente (no caso do `Maximize`) ou inferiormente (no caso do `Minimize`).
1. Interpretar os resultados no caso do resultado ser `pywraplp.Solver.OPTIMAL`. Para saber o valor de uma variável pode ser usado o método `solution_value`, que devolve sempre um `double`. Também é possível aceder à função objectivo com o método `Objective` e ao respectivo valor com o método `Value`. Para converter o valor do tipo `double` para um `int` deve ser usada a função `round`, pois o valor pode sofrer de pequenos erros de precisão.

Por exemplo, o programa seguinte tenta encontrar $x$ e $y$ que minimizem $3x+4y$ satisfazendo as seguintes restrições:

$$
\left\{
\begin{array}{l}
5x + 6y \ge 11\\
7x + 5y \ge 5\\
x \ge 0\\
y \ge 0
\end{array}
\right.
$$

In [16]:
# Importar biblioteca
from ortools.linear_solver import pywraplp
# Criar instância do solver
solver = pywraplp.Solver.CreateSolver('SCIP')
# Adicionar variáveis
x = solver.IntVar(0.0,solver.infinity(),"x")
y = solver.IntVar(0.0,solver.infinity(),"y")
# Adicionar restrições
solver.Add(5*x + 6*y >= 11)
solver.Add(7*x + 5*y >= 5)
# Definir objectivo
solver.Minimize(3*x + 4*y)
# Invocar o solver
status = solver.Solve()
# Interpretar os resultados
print ("x = ",round(x.solution_value()))
print ("y = ",round(y.solution_value()))
print ("objectivo = ",round(solver.Objective().Value()))

x =  1
y =  1
objectivo =  7


Acrescente as restrições $y=0$, $x \leq 2$. Qual é o resultado?

In [17]:
# completar

solver.Add(y==0)
solver.Add(x<=2)

status = solver.Solve()

print ("x = ",round(x.solution_value()))
print ("y = ",round(y.solution_value()))
print ("objectivo = ",round(solver.Objective().Value()))

# The solution does not make sense (the restraints are not satisfied)! Check status:
if status==pywraplp.Solver.INFEASIBLE:
    print("The problem is not feasible.")
if status==pywraplp.Solver.FEASIBLE:
    print("The problem is feasible.")
if status==pywraplp.Solver.OPTIMAL:
    print("The solution is optimal.")

x =  0
y =  0
objectivo =  0
The problem is not feasible.


Em alternativa aos métodos `Add` e `Minimize`, é possível utilizar os métodos `Constraint`, `SetCoefficient`, `Objective` e `SetMinimization` ou `SetMaximization`. Reescreva o código anterior utilizando estes últimos.

In [18]:
# completar
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')
# Adicionar variáveis
x = solver.IntVar(0.0,solver.infinity(),"x")
y = solver.IntVar(0.0,solver.infinity(),"y")

c1 = solver.Constraint(11, solver.infinity(), "c1")
c1.SetCoefficient(x, 5)
c1.SetCoefficient(y, 6)

c2 = solver.Constraint(5, solver.infinity(), "c2")
c2.SetCoefficient(x, 7)
c2.SetCoefficient(y, 5)

objective = solver.Objective()
objective.SetCoefficient(x, 3)
objective.SetCoefficient(y, 4)
objective.SetMinimization()

status = solver.Solve()

if status==pywraplp.Solver.INFEASIBLE:
    print("The problem is not feasible.")
if status==pywraplp.Solver.FEASIBLE:
    print("The problem is feasible.")
if status==pywraplp.Solver.OPTIMAL:
    print("The solution is optimal.")
    print ("x = ",round(x.solution_value()))
    print ("y = ",round(y.solution_value()))
    print ("objectivo = ",round(solver.Objective().Value()))

The solution is optimal.
x =  1
y =  1
objectivo =  7


Reverta o sentido da primeira restrição, i.e., imponha $5x + 6y \le 11$, mantendo as restantes. Qual é a nova solução?

Nota: poderá ser útil usar os métodos `Clear` e `SetBounds`  da classe `Constraint`.

In [19]:
# completar

c1.Clear()

c1.Clear()
c1.SetBounds(-solver.infinity(), 11)

status = solver.Solve()

print ("x = ",round(x.solution_value()))
print ("y = ",round(y.solution_value()))
print ("objectivo = ",round(solver.Objective().Value()))

if status==pywraplp.Solver.INFEASIBLE:
    print("The problem is not feasible.")
if status==pywraplp.Solver.FEASIBLE:
    print("The problem is feasible.")
if status==pywraplp.Solver.OPTIMAL:
    print("The solution is optimal.")

x =  1
y =  0
objectivo =  3
The solution is optimal.
